In [13]:
from conga2 import *
import numpy as np
import collections as co

from igraph import *
import operator
import itertools
import argparse

import random

In [30]:
teams = 10
players_per_team = 10
min_stayed = 0.7 * players_per_team
transfer_windows = 10

In [31]:
def GenerateFootballGraph(teams, players_per_team, min_stayed, transfer_windows):

    #Generate team_labels
    team_labels = []
    for i in range(0,teams):
        lst = [*range(i*players_per_team, (i+1)*players_per_team)]
        team_labels.append(lst)

    #Generate graph
    test_graph = Graph()
    test_graph.add_vertices(teams*players_per_team)
    for t in range(0,teams):
        for i in range(0,players_per_team):
            for j in range(i,players_per_team-1):
                test_graph.add_edges([(i + t * players_per_team ,j+ 1 + t * players_per_team)])


    #Make transfers
    for times in range(0,transfer_windows):
        for i in range(0,teams-1):
            players_chosen = int((len(team_labels[i]) - min_stayed) / 2)
            for j in range(0,players_chosen):
                player_to_transfer = (team_labels[i][random.randint(0, len(team_labels[i]) - 1)])
                team_labels[i].remove(player_to_transfer)
                new_team = random.randint(0,teams-1)
                if new_team == i:
                    new_team += 1
                    new_team %= teams
                team_labels[new_team].append(player_to_transfer)
                for k in team_labels[new_team]:
                    test_graph.add_edges([(player_to_transfer,k)])
    #Graph.simplify(test_graph)
    return test_graph

In [32]:
g = GenerateFootballGraph(teams, players_per_team, min_stayed, transfer_windows)
len(g.components())
print(g)

IGRAPH U--- 100 1522 --
+ edges:
  0 --   0   0   0   0   1   2   3   4   5   6   7   8   9  12  14  20  21  22
23  25  26  27  28  29  32  55  60  61  62  62  63  64  65  66  67  68  69  81
89
  1 --   0   2   3   4   5   6   7   8   9  16  21  25  29  65  74  86
  2 --   0   1   2   2   3   3   4   5   6   7   8   9  12  13  22  61  64  65
65  66  67  67  68  69  77  86  86
  3 --   0   1   2   2   3   3   3   3   4   5   6   7   8   9  12  21  22  46
49  51  60  61  61  62  63  64  65  65  66  67  68  69  70  71  75  76  77  78
79  86
  4 --   0   1   2   3   5   6   7   8   9  16  21  25  29  65  74  86
  5 --   0   1   2   3   4   5   5   6   7   8   9  25  28  29  30  36  37  43
51  59  60  67  73  74  81  86
  6 --   0   1   2   3   4   5   7   8   9  16  21  25  29  65  74  86
  7 --   0   1   2   3   4   5   6   8   9  16  21  25  29  65  74  86
  8 --   0   1   2   3   4   5   6   7   9  16  21  25  29  65  74  86
  9 --   0   1   2   3   4   5   6   7   8   9   9  15  17  18

In [33]:
result = conga(g,optimal_count=5)

In [34]:
result.pretty_print_cover(result.optimal_count, label="CONGA_index")


Community 0:
	 0
	 1
	 2
	 3
	 4
	 5
	 6
	 7
	 8
	 10
	 11
	 12
	 13
	 14
	 16
	 19
	 20
	 21
	 22
	 23
	 24
	 25
	 26
	 27
	 28
	 29
	 30
	 32
	 33
	 34
	 35
	 36
	 37
	 38
	 39
	 40
	 41
	 42
	 43
	 44
	 45
	 46
	 47
	 48
	 49
	 50
	 51
	 54
	 55
	 56
	 57
	 58
	 59
	 60
	 61
	 62
	 63
	 65
	 66
	 67
	 68
	 69
	 70
	 71
	 73
	 74
	 75
	 76
	 78
	 79
	 80
	 83
	 85
	 86
	 87
	 88
	 89
	 86
	 21
	 77
	 72
	 64
	 9
	 31
	 15
	 84
	 18
	 17
	 52
	 53
	 86
	 74
	 12
	 81
	 29
	 46
	 34
	 89
	 85
	 14
	 20
	 81
	 79
	 72
	 38
	 33
	 82
	 82
	 86
	 86
	 86
	 81
	 81

Community 1:
	 9
	 15
	 17
	 18
	 31
	 52
	 53
	 64
	 72
	 77
	 84
	 90
	 91
	 92
	 93
	 94
	 95
	 96
	 97
	 98
	 99
	 77
	 72
	 64
	 9
	 31
	 15
	 84
	 18
	 17
	 52
	 53
	 77
	 77
	 72
	 72

Community 2:
	 81
	 81
	 81
	 81
	 81

Community 3:
	 82
	 82
	 82
	 82
	 82

Community 4:
	 86
	 86
	 86
	 86



In [4]:
from time import sleep
from typing import Counter
from igraph import *
import igraph as ig
import csv
import collections
import random

def delete_comments(csvfile):
    for row in csvfile:
        raw = row.split('#')[0].strip()
        if raw:
            yield raw

def read_graph( edges_file):
    print('reading from file')
    edges = []
   

    with open(edges_file , 'r') as csvfile:
        csvreader = csv.reader(delete_comments(csvfile), delimiter=',')
        edges = list(csvreader)

    print('edges setup')
    edgevertices = list([edge[0] for edge in edges] + [edge[1] for edge in edges])

    print('edgevertices')
    edgevertices = list(dict.fromkeys(edgevertices))
  
    print('inverse_indicies')
    inverse_indicies = dict(zip(edgevertices, range(len(edgevertices))))

    g = Graph()

    g.add_vertices(len(edgevertices))
    print('no vertices   ' + str(len(edgevertices)))
    g.vs["id"] = [vertex for vertex in edgevertices]
    g.vs["label"] = g.vs["id"]
    g.vs["vertex_size"] = 8
    g.vs["newlpalabel"] = 0
    g.es["weight"] = 2
    g.vs["vertex_color"] = 0

    for vert in g.vs:
        vert["vertex_color"] = f'rgb({random.randint(0, 255)}, {random.randint(0, 255)}, {random.randint(0, 255)})'
    g.vs["lpalabel"] = list(zip(range(len(edgevertices)), g.vs["vertex_color"]))

    print("adding edges")
    try:
        for edge in edges:
            g.add_edge(inverse_indicies[edge[0]], inverse_indicies[edge[1]])
    except:
        print("Error occured during graph creating")
        quit()

    g.es["weigth"] = 1
    print("graph created")
    return g

In [5]:
swiss = read_graph("..\scrapers\Swiss-Super-League-StatsEdges.csv")

reading from file
edges setup
edgevertices
inverse_indicies
no vertices   585
adding edges
graph created


In [14]:
Graph.simplify(swiss)
SwissLeague = conga(swiss,optimal_count=10)

In [37]:
#SwissLeague.pretty_print_cover(SwissLeague.optimal_count, label="CONGA_index")

cover = SwissLeague._covers[10]

list_communities = []
for i in range(0, 10):
    list_communities.append(cover._clusters[i])

print(list_communities)
len(list_communities)

[[0, 1, 2, 4, 5, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 247, 248, 251, 252, 253, 254, 319, 320, 321, 322, 323, 324, 399, 400, 401, 402, 403, 404, 405, 440, 441, 442, 443, 444, 445, 446, 447, 565, 576, 249, 250, 65, 7, 97, 439, 72, 3, 6], [3, 6, 60, 63, 64, 66, 67, 68, 69, 71, 72, 74, 76, 77, 78, 79, 80, 170, 220, 221, 223, 224, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 306, 307, 308, 309, 310, 311, 371, 372, 373, 374, 375, 376, 377, 471, 472, 473, 474, 475, 476, 477, 508, 509, 510, 511, 512, 513, 514, 567, 570, 573, 577, 580, 61, 70, 207, 73, 112, 65, 222, 62, 225, 226, 75], [7, 182, 183, 184, 185, 186, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 231, 232, 233, 234, 235, 237, 238, 239, 240, 241, 187, 146, 181, 236], [19, 61, 131, 414, 415, 416, 417, 418, 419, 420, 422, 423, 426, 427, 470, 478, 479, 480, 481, 482, 540, 541, 542, 543, 544, 545, 546, 547, 548, 578, 583, 410, 425, 153, 155, 394, 421, 483, 46,

10

In [42]:
croatia = read_graph("..\scrapers\cr-StatsEdges.csv")
Graph.simplify(croatia)
print(croatia)

reading from file
edges setup
edgevertices
inverse_indicies
no vertices   571
adding edges
graph created
IGRAPH U--- 571 8941 --
+ attr: id (v), label (v), lpalabel (v), newlpalabel (v), vertex_color (v),
  vertex_size (v)
+ edges:
  0 --   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
19  46  92 141 200 201 202 203 204 205 206 207 208 209 263 264 304 305 306 307
308 309 369 370 371 372 373 374 406 461 462 463 464 465 466 467 468 469 549
  1 --   0   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
19  92 141 200 201 202 203 204 205 206 207 208 209 263 264 304 305 306 307 308
309 369 370 371 372 373 374 549
  2 --   0   1   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
19  21  71  97 134 200 201 202 203 204 205 206 207 208 209 220 314 316 319 373
375 376 377 378 379 380 381 470 471 472 549
  3 --   0   1   2   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
19  60  80 157 158 160 163 164 166 169 172 174 275 276 277 278 

In [43]:
Graph.simplify(croatia)
CroatiaLeague = conga(croatia,optimal_count=10)

In [ ]:
cover = CroatiaLeague._covers[10]

list_communities = []
for i in range(0, 10):
    list_communities.append(cover._clusters[i])

#print(list_communities)
#len(list_communities)
SwissLeague.pretty_print_cover(SwissLeague.optimal_count, label="CONGA_index")

In [108]:
color_dict = {"0":"white","1": "blue", "2": "pink","3":"red","4":"black","5":"pink","6":"yellow","7":"orange","8":"green","9":"blue"}
num = -1
for v in swiss.vs:
     v["color"] = 0
     num += 1
     li = 0
     for list in list_communities:
          if(num in list):
               v["color"] = color_dict[str(li)]
               break
          li += 1

for v in swiss.vs:
     print(v["color"])


plot(swiss)

white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
yellow
white
yellow
yellow
yellow
white
yellow
yellow
yellow
red
pink
yellow
yellow
yellow
yellow
yellow
yellow
yellow
yellow
red
red
red
red
red
red
blue
white
red
red
red
red
red
red
red
red
red
red
red
red
red
blue
red
red
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
pink
pink
pink
blue
pink
pink
pink
white
pink
pink
pink
white
pink
pink
pink
pink
white
black
black
black
black
black
blue
black
black
black
black
black
black
black
black
black
black
black
black
black
black
black
pink
pink
pink
pink
pink
pink
pink
pink
blue
pink
pink
pink
blue
pink
pink
white
pink
pink
pink
pink
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
white
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
blue
red
red
red
red
red
red
red
r